In [ ]:
import tweepy
import time
from datetime import datetime, date
import math

# Twitter API autentifikācija
client = tweepy.Client(
    bearer_token="AAAAAAAAAAAAAAAAAAAAAALO0AEAAAAAhBaYZUgl1UGeV56RDdsZUejn96Y%3D85Uw64NlwW5lnbKed7MYcP0ZbbLXSk3nXDKTf1sd065Npogryh",
    wait_on_rate_limit=True  # Automātiski gaida, ja sasniegts API pieprasījumu limits
)

# 📌 Analizē lietotāja kontu
def AnalyseAccount(user):
    created_at = user.created_at
    total_days = (date.today() - created_at.date()).days
    weight = 0

    if total_days < 31:
        weight += total_days
    else:
        if user.public_metrics["followers_count"] < 100:
            weight += 10

    return weight

# Analizē lietotāja tweetus (unikālo saturu)
def AnalyseTweets(user_id, tweet_limit):
    time.sleep(2)  # Pauze, lai izvairītos no API limita
    tweets = client.get_users_tweets(id=user_id, max_results=tweet_limit).data
    unique_tweets = set()
    tweets_analysed = 0

    if tweets:
        for tweet in tweets:
            unique_tweets.add(tweet.text)
            tweets_analysed += 1

        return 100 * (tweets_analysed - len(unique_tweets)) / tweets_analysed
    return 0

#  Nosaka, vai lietotājs ir bots
def BotScore(username, tweet_limit):
    time.sleep(2)  # Pauze, lai izvairītos no API limita
    user = client.get_user(username=username, user_fields=["created_at", "public_metrics", "username"]).data

    if not user:
        print(f"Lietotājs {username} nav atrasts.")
        return False  # Ja lietotājs nav atrasts

    total_score = 0
    total_score += AnalyseAccount(user)
    total_score += AnalyseTweets(user.id, tweet_limit)

    print(f"Bot Score for {username}: {total_score}")

    return total_score > 85  # Ja virs 85, iespējams bots

# Testējam botu detekciju
print(BotScore("jenkinsci", 50))
